# Test whether writing a `frcmod` file from ParmEd after setting up an OpenMM `System` with smirnoff99Frosst parameters can be successfully used with `tleap`

In [131]:
import numpy as np
import parmed as pmd
from openforcefield.typing.engines.smirnoff import ForceField, unit

In [132]:
from openeye.oechem import (
    oemolistream, oemolostream, OEIFlavor_MOL2_Forcefield,
    OEIFlavor_Generic_Default, OEIFlavor_PDB_Default, OEIFlavor_PDB_ALL,
    OEFormat_MOL2, OEFormat_MOL2H, OEWriteMolecule, OETriposAtomNames, OEMol,
    OEFormat_PDB, OESmilesToMol, OEAddExplicitHydrogens, OEHasAtomIdx,
OEAtomGetResidue)

## Read in the host and guest molecules to `OEMol`s

In [133]:
filename = 'MGO-sybyl.mol2'
molecules = []

ifs = oemolistream()
flavor = OEIFlavor_MOL2_Forcefield
ifs.SetFlavor(OEFormat_MOL2, flavor)
ifs.open(filename)

for mol in ifs.GetOEMols():
    OETriposAtomNames(mol)
    molecules.append(OEMol(mol))
    
filename = 'MOL-sybyl.mol2'
ifs = oemolistream()
flavor = OEIFlavor_MOL2_Forcefield
ifs.SetFlavor(OEFormat_MOL2, flavor)
ifs.open(filename)

for mol in ifs.GetOEMols():
    OETriposAtomNames(mol)
    molecules.append(OEMol(mol))

## Read the in the topology information from a PDB

In [134]:
filename = 'smirnoff.pruned.pdb'
pdb = pmd.load_file(filename)
topology = pmd.Structure()
topology += pdb.split()[0][0]
topology += pdb.split()[1][0]

In [135]:
topology

<Structure 143 atoms; 7 residues; 148 bonds; NOT parametrized>

In [160]:
topology.write_pdb('MGO-MOL.pdb')

## Create the OpenMM `System`

In [136]:
ff = ForceField('forcefield/smirnoff99Frosst.offxml')
system = ff.createSystem(
    topology.topology,
    molecules,
    nonbondedCutoff=1.1 * unit.nanometer,
    ewaldErrorTolerance=1e-4)

The OpenMM object has atom names, but not atom types.

In [137]:
for atom in [x for _, x in zip(range(5), topology.topology.atoms())]:
    print(atom.name)

C1
H1
O1
C2
H2


In [143]:
for atom in [x for _, x in zip(range(5), topology.topology.atoms())]:
    print(atom.type)

AttributeError: 'Atom' object has no attribute 'type'

## Load into a ParmEd `Structure`

In [144]:
hg_structure = pmd.openmm.topsystem.load_topology(topology.topology, system, topology.positions)

In [145]:
hg_parm = pmd.amber.AmberParm.from_structure(hg_structure)

At this point, the atom types are numeric.

In [146]:
for atom in [x for _, x in zip(range(5), hg_parm.atoms)]:
    print(atom.type)

1
2
3
4
5


In fact, I can't even save the `AmberParm` directly as a `frcmod`, because it tries to loop through the atom types.

In [147]:
pmd.amber.AmberParameterSet.write(hg_parm, dest='smirnoff.frcmod', style='frcmod')

AttributeError: 'AmberParm' object has no attribute 'atom_types'

If I save the `Structure`, then I can read the `prmtop` in as an `AmberParm`, re-initialize that with some magic that I don't completely understand, and then save a `frcmod`.

In [148]:
hg_structure.save('smirnoff.prmtop')

OSError: smirnoff.prmtop exists; not overwriting

In [75]:
hg_parm = pmd.amber.AmberParm('smirnoff.prmtop')

In [79]:
parameter_set = pmd.amber.AmberParameterSet.from_structure(hg_parm)

In [80]:
parameter_set.write('smirnoff.frcmod')

## Okay, now let's do our thing and re-write the atom types with 2 character strings to recreate the `tleap` error...

We need 17 atoms for the guest and 126 for the host (giving every atom a unique type).

In [112]:
guest_types = [f'G{i}' for i in range(10)] + [f'g{i}' for i in range(7)]

In [117]:
assert len(guest_types) >= molecules[1].GetMaxAtomIdx()

In [104]:
host_types = [f'{i}{j}' for j in range(10) for i in ['A', 'a', 'B', 'b', 'C', 'c', 'D', 'd', 'E', 'e',
                                                    'F', 'f', 'I']]

In [118]:
assert len(host_types) >= molecules[0].GetMaxAtomIdx()

There are a few ways to do this, but as a shortcut, I know the host atoms come before the guest atoms and I know how many atoms we have in each molecule, so...

In [128]:
new_atom_types = host_types[0:molecules[0].GetMaxAtomIdx()] + guest_types

In [129]:
assert len(new_atom_types) == len(hg_structure.atoms)

In [150]:
structure = pmd.load_file('smirnoff.prmtop', structure=True)
# If I don't re-read in the `structure`, it won't work. Completely unclear why.


for index, atom in enumerate(structure.atoms):
    atom.type = new_atom_types[index]
    structure.parm_data['AMBER_ATOM_TYPE'][index] = new_atom_types[index]

In [151]:
structure.load_atom_info()
structure.fill_LJ()

In [152]:
new_parameters = pmd.amber.AmberParameterSet.from_structure(structure)
new_parameters.write('smirnoff-unique.frcmod')

## Write host and guest `mol2` files with the new atom types...

In [156]:
for residue in structure.residues:
    print(residue.name)

MGO
MGO
MGO
MGO
MGO
MGO
MOL


In [157]:
single_guest = structure[':7']
single_guest.save('MOL-unique.mol2', overwrite=True)

single_host = structure[':1']
single_host.save('MGO-unique.mol2', overwrite=True)

## Write a `tleap` input file

In [162]:
with open('tleap.in', 'w') as file:
    tleap_header = f'''\n
loadamberparams smirnoff-unique.frcmod
MGO = loadmol2 MGO-unique.mol2
MOL = loadmol2 MOL-unique.mol2

model = loadpdb MGO-MOL.pdb
bond model.1.O1 model.2.C4
bond model.2.O1 model.3.C4
bond model.3.O1 model.4.C4
bond model.4.O1 model.5.C4
bond model.5.O1 model.6.C4
bond model.6.O1 model.1.C4

check model
saveamberparm smirnoff-unique.prmtop smirnoff-unique.rst7
'''
    file.write(tleap_header)